# Intermediate Machine Learning

## Manejando los NaN o valores vacíos

Una vez que se ha hecho una introducción al Machine Learning, se van a dar pasos para mejorar nuestros modelos. Se comenzará explicando cómo combatir los valores vacíos.

Es muy común encontrarnos valores vacíos en nuestro conjunto de datos. La mayoría de las librerías de Machine Learning, incluendo a `scikit-learn`, evolverán un error si se intenta construir un modelo con valores vacíos, así que hay que buscar una estrategia para trabajar con ellos:

- **Eliminar las columnas con valores vacíos**. La opción más simple para eliminar los valores vacíos es, directamente eliminar la columna con valores vacíos. Sin embargo, esta opción no es aconsejable salvo que la mayoría de los valores de la columna estén vacíos, ya que se pierder mucha información potencialmente útil.
![Drop](https://i.imgur.com/Sax80za.png)

- **Imputación de resultados**. Consiste en imputar un valor a las celdas vacíos de una columna. Por ejemplo, con la media. Aunque el valor imputado no será exacto en la mayoría de los casos, generalmente nos permitirá obtener modelos más adecuados que eliminando la columna entera. Salvo que falten muchos valores vacíos, generalmente funciona bastante bien la imputación.
![Imputation](https://i.imgur.com/4BpnlPA.png)

- **Una extensión a la imputación**. El problema que tiene la imputación es que algunos valores estarán por encima o por debajo de la realidad, pudiendo crear filas únicas con resultados que no tienen mucho sentido. Por ello, en ocasiones es interesante añadir una columna indicando si el valor faltaba o no y se ha imputado. En ocasiones esto mejora los resultados y en otras ocasiones, no funciona en absoluto.
![Imputation](https://i.imgur.com/UWOyg4a.png)

### Ejemplo de eliminación e imputación

In [6]:
import os

os.getcwd()
os.chdir("C:\\Users\\jmoreno7\\Downloads\\")

In [12]:
import pandas as pd
from sklearn.model_selection import train_test_split

#Se cargan los datos
X_full = pd.read_csv("train.csv", index_col="Id")
X_test_full = pd.read_csv("test.csv", index_col="Id")

#Se eliminan las líneas con NaN en el target o variable Y.
X_full.dropna(axis=0, subset=["SalePrice"], inplace=True)

#Se selecciona la variable target
y = X_full.SalePrice

#Se elimina la variable price del predictor
X_full.drop(["SalePrice"],axis=1, inplace=True)

#Para hacerlo más simple, ns quedamos solo con los operadores numéricos
X = X_full.select_dtypes(exclude=["object"])
X_test = X_test_full.select_dtypes(exclude=["object"])

#Realizamos el train
X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2, random_state=0)

In [13]:
#Realizamos un análisis preliminar
X_train.head()

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold
Id,,,,,,,,,,,,,,,,,,,,,
619,20,90.0,11694,9,5,2007,2007,452.0,48,0,...,774,0,108,0,0,260,0,0,7,2007
871,20,60.0,6600,5,5,1962,1962,0.0,0,0,...,308,0,0,0,0,0,0,0,8,2009
93,30,80.0,13360,5,7,1921,2006,0.0,713,0,...,432,0,0,44,0,0,0,0,8,2009
818,20,NaN,13265,8,5,2002,2002,148.0,1218,0,...,857,150,59,0,0,0,0,0,7,2008
303,20,118.0,13704,7,5,2001,2002,150.0,0,0,...,843,468,81,0,0,0,0,0,1,2006


In [14]:
#Contamos dimensiones
X_train.shape

(1168, 36)

In [21]:
#Contamos NaN
missing_values = X_train.isna().sum()
missing_values

MSSubClass         0
LotFrontage      212
LotArea            0
OverallQual        0
OverallCond        0
YearBuilt          0
YearRemodAdd       0
MasVnrArea         6
BsmtFinSF1         0
BsmtFinSF2         0
BsmtUnfSF          0
TotalBsmtSF        0
1stFlrSF           0
2ndFlrSF           0
LowQualFinSF       0
GrLivArea          0
BsmtFullBath       0
BsmtHalfBath       0
FullBath           0
HalfBath           0
BedroomAbvGr       0
KitchenAbvGr       0
TotRmsAbvGrd       0
Fireplaces         0
GarageYrBlt       58
GarageCars         0
GarageArea         0
WoodDeckSF         0
OpenPorchSF        0
EnclosedPorch      0
3SsnPorch          0
ScreenPorch        0
PoolArea           0
MiscVal            0
MoSold             0
YrSold             0
dtype: int64

In [22]:
#Devolvemos las columnas solo con NaN
missing_values[missing_values>0]

LotFrontage    212
MasVnrArea       6
GarageYrBlt     58
dtype: int64

In [23]:
total_nan = sum(missing_values[missing_values>0])

276

In [ ]:
#How many rows are in the training data?
num_rows = 1168

#How many columns in the training data have missing values?
num_cols_with_missing = 3

#How many missing entries are contained in all of the training data?
tot_missing = 276

Teniendo en cuenta todo esto, el mejor acercamiento posible no será jamás eliminar datos, ya que hay relaticamente pocos NaN y no llegan al 20% del total de datos.

En cualquier caso, vamos a comparar reasultados eliminando datos y dejando

In [25]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

# Function for comparing different approaches
def score_dataset(X_train, X_valid, y_train, y_valid):
    model = RandomForestRegressor(n_estimators=100, random_state=0)
    model.fit(X_train, y_train)
    preds = model.predict(X_valid)
    return mean_absolute_error(y_valid, preds)

### Ejemplo eliminando los datos

In [27]:
# Get names of columns with missing values
X_train.columns

Index(['MSSubClass', 'LotFrontage', 'LotArea', 'OverallQual', 'OverallCond',
       'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2',
       'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF',
       'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath',
       'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd', 'Fireplaces',
       'GarageYrBlt', 'GarageCars', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF',
       'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'MiscVal',
       'MoSold', 'YrSold'],
      dtype='object')

In [28]:
# Drop columns in training and validation data
reduced_X_train = X_train.drop(["LotFrontage","MasVnrArea","GarageYrBlt"], axis = 1)
reduced_X_valid = X_valid.drop(["LotFrontage","MasVnrArea","GarageYrBlt"], axis = 1)

In [29]:
#Getting Score DataSet
print("MAE (Drop columns with missing values):")
print(score_dataset(reduced_X_train, reduced_X_valid, y_train, y_valid))

MAE (Drop columns with missing values):
17837.82570776256


### Ejemplo relizando una imputación
En el ejemplo de debajo, se imputarán valores usando la media de cada columna

In [31]:
from sklearn.impute import SimpleImputer

# imputation
my_imputer = SimpleImputer()
imputed_X_train = pd.DataFrame(my_imputer.fit_transform(X_train))
imputed_X_valid = pd.DataFrame(my_imputer.fit_transform(X_valid))

# imputation removed column names; put them back
imputed_X_train.columns = X_train.columns
imputed_X_valid.columns = X_valid.columns

print("MAE (Imputation):")
print(score_dataset(imputed_X_train, imputed_X_valid, y_train, y_valid))

MAE (Imputation):
18056.85163242009


#### Comparando resultados
¿Hay algo sorprendente en ls resultados? ¿Por qué piensas que un acercamiento ha sido mejor que el otro?

El resultado ha sido a priori sorprendente. Debido a que solo hay unos pocos *missing values*, era de esperar un mejor resultado imputando, pero ha sido al revés.

Aunque esto se puede atribuir a que haya algo de ruido en los datos del dataset, también se puede pensar que la imputación no es buena para este dataset. Por ello, en vez de rellenar cada valor con la media, podría tener sentido rellenarlo con 0, con la mediana o con la moda (el valor más repetido), o usar otro método. Por ejemplo, si se tiene en consideración la columna ``GarageYrBlt``, es probable que el valor NaN indique que la casa no tiene garaje. Por eso se podría rellenar con 0, el valor mínimo, etc.

### Resumen de lo explicado
Se va a repetir el proceso, usando como imputer la Mediana:

In [4]:
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.impute import SimpleImputer


os.getcwd()
os.chdir("C:\\Users\\jmoreno7\\Downloads\\")

#Se cargan los datos
X_full = pd.read_csv("train.csv", index_col="Id")
X_test_full = pd.read_csv("test.csv", index_col="Id")

####Preparamos el train
#Se eliminan las líneas con NaN en el target o variable Y.
X_full.dropna(axis=0, subset=["SalePrice"], inplace=True)

#Se selecciona la variable target
y = X_full.SalePrice

#Se elimina la variable price del predictor
X_full.drop(["SalePrice"],axis=1, inplace=True)

#Para hacerlo más simple, ns quedamos solo con los operadores numéricos
X = X_full.select_dtypes(exclude=["object"])

####Preparamos el test
#Nos quedamos solo con las variable snuméricas en el test
X_test = X_test_full.select_dtypes(exclude=["object"])

#Realizamos el train sobre los datos cargados "train"
X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2, random_state=0)


###Después de las pruebas realizadas, vamos a hacerlo badándonos en la mediana
#Transformamos los NaN por la mediana
my_imputer = SimpleImputer(strategy='median')
final_X_train = pd.DataFrame(my_imputer.fit_transform(X_train))
final_X_valid = pd.DataFrame(my_imputer.transform(X_valid))

final_X_train.columns = X_train.columns
final_X_valid.columns = X_valid.columns

#Y también aplicamos las mismtas tranformaciones sobre el test
final_X_test = pd.DataFrame(my_imputer.transform(X_test))

#Definimos el modelo
model = RandomForestRegressor(n_estimators=100, random_state=0)
model.fit(final_X_train, y_train)

#Realizamos predicciones sobre el test data
preds_test = model.predict(final_X_test)